<a href="https://colab.research.google.com/github/mathluva/Bert-Toxic-Comments/blob/main/Toxic_Comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [229]:
!nvidia-smi

Sun Feb 28 08:40:40 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P0    30W /  70W |  11406MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
import pandas as pd

import numpy as np
import torch
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import files
uploaded = files.upload()

ModuleNotFoundError: No module named 'google.colab'

In [3]:
import io
df = pd.read_csv("../data/train.csv")
df.head()

,exercise_text,三角函数,函数奇偶性,导数,平面向量,数列,逻辑与命题关系,集合
0,已知函数其中曲线在点处的切线垂直于轴求的值求函数的极值求导得曲线在点处的切线垂直于轴则函数在...,0,0,1,0,0,0,0
1,已知函数的图像在点处的切线斜率为类猜想等数学思想,0,0,1,0,0,0,0
2,设函数给出如下四个命题若则为奇函数若则函数在上是增函数函数的图象关于点成中心对称图形关于的方...,0,1,0,0,0,1,0
3,若平面向量与的夹角是且则的坐标为试题分析因为所以所以所以考点向量的共线向量的模,0,0,0,1,0,0,0
4,分已知函数是偶函数求的值设若函数与的图象有且只有一个公共点求实数的取值范围当时由题得方程有一...,0,1,0,0,0,0,0


In [4]:
rows = len(df)
print(f'There are {rows} rows')

There are 2977 rows


In [234]:
ids = df.id.nunique()
print(f'There are {ids} unique ids')

There are 159571 unique ids


In [5]:
#replace index with id
df.set_index('id', inplace =True) 

KeyError: "None of ['id'] are in the columns"

In [236]:
#toxicity is total of all categories per comment
# toxicity 6 means comment was classified for all categories
df.toxicity.value_counts() 

0.0    143346
1.0      6360
3.0      4209
2.0      3480
4.0      1760
5.0       385
6.0        31
Name: toxicity, dtype: int64

In [237]:
#keep comments with only one label
df = df[df['toxicity'].values ==1]
df

,comment_text,identity_hate,insult,obscene,set,severe_toxic,threat,toxic,toxicity
id,,,,,,,,,
0005c987bdfc9d4b,hey what is it talk what is it an exclusive ...,0.0,0.0,0.0,train,0.0,0.0,1.0,1.0
0007e25b2121310b,bye don t look come or think of comming back...,0.0,0.0,0.0,train,0.0,0.0,1.0,1.0
001956c382006abd,i m sorry i m sorry i screwed around with some...,0.0,0.0,0.0,train,0.0,0.0,1.0,1.0
002264ea4d5f2887,why can t you believe how fat artie is did yo...,0.0,0.0,0.0,train,0.0,0.0,1.0,1.0
005f59485fcddeb0,sorry puck but no one ever said dick was numb...,0.0,0.0,0.0,train,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...
fcf5a6ad5918f164,your boring and retard ed two,0.0,0.0,0.0,train,0.0,0.0,1.0,1.0
fd0129fde97321cb,why did that idiot revert the reversion i made...,0.0,0.0,0.0,train,0.0,0.0,1.0,1.0
fd68ef478b3dfd05,ps you re all middle aged losers at home in yo...,0.0,0.0,0.0,train,0.0,0.0,1.0,1.0


In [238]:
#check sum of identity_hate
df.identity_hate.value_counts()

0.0    6306
1.0      54
Name: identity_hate, dtype: int64

In [239]:
#create new column label
#decode the rows
for index, row in df.iterrows():
    if df.loc[index, 'identity_hate'] == 1:
      df.loc[index, 'label'] = 'identity_hate'

    elif df.loc[index, 'insult'] == 1:
      df.loc[index, 'label'] = 'insult'

    elif df.loc[index, 'obscene'] == 1:
      df.loc[index, 'label'] = 'obscene'

    elif df.loc[index, 'severe_toxic'] == 1:
      df.loc[index, 'label'] = 'severe_toxic'

    elif df.loc[index, 'threat'] == 1:
      df.loc[index, 'label'] = 'threat'

    else:
        df.loc[index, 'label'] = 'toxic'

In [240]:
#create new column to hold label value
possible_lables = ['identity_hate', 'insult', 'obscene', 'severe_toxic', 'threat', 'toxic']

label_dict = {}
for index, label in enumerate(possible_labels):
    label_dict[label] = index

df['label_value'] = df.label.replace(label_dict)

In [6]:
df.tail(10)

,exercise_text,三角函数,函数奇偶性,导数,平面向量,数列,逻辑与命题关系,集合
2967,在平面上若则的取值范围是试题分析当时此时点与点重合此时是边长为的正方形所以当时因为且所以为矩...,0,0,0,1,0,0,0
2968,设若函数有大于零的极值点则的取值范围是试题分析因为是单调递增的函数有大于零的极值点即考点导函...,0,0,1,0,0,0,0
2969,在下列命题中若向量共线则向量所在的直线平行若向量所在的直线为异面直线则向量一定不共面若三个向...,0,0,0,1,0,1,0
2970,如图在中分别为边的中点析为的中点考点平面向量的基底表示,0,0,0,1,0,0,0
2971,已知公差不为的等差数列满足成等比数列项和则的值为,0,0,0,0,1,0,0
2972,对任意正整数定义双阶乘如下当为偶数时当为奇数时现有如下四个命题设若的个位数不是则设为正质数为...,0,0,0,0,0,1,0
2973,设函数是定义在上的函数其中的导函数满足对于恒成立则试题分析函数的导数为故函数是定义在上的减函...,0,0,1,0,0,0,0
2974,设求的单调区间并证明对上的任意都有将的图像向下平移个单位同时将的图像向上平移个单位使它们恰有...,0,0,1,0,0,0,0
2975,已知数列的前项和为则关于的命题其中若是关于的二次函数则是等差数列若是等比数列且则若是等差数列...,0,0,0,0,1,1,0
2976,已知函数的导函数为的图象在点处的切线方程为且直线是函数的图象的一条切线考点导数的几何意义利用...,0,0,1,0,0,0,0


In [243]:
#check totals
df['label'].value_counts()

toxic            5666
obscene           317
insult            301
identity_hate      54
threat             22
Name: label, dtype: int64

In [244]:
df = df.drop(columns= ['set', 'toxicity'])
df.head()

,comment_text,identity_hate,insult,obscene,severe_toxic,threat,toxic,label,label_value
id,,,,,,,,,
0005c987bdfc9d4b,hey what is it talk what is it an exclusive ...,0.0,0.0,0.0,0.0,0.0,1.0,toxic,5
0007e25b2121310b,bye don t look come or think of comming back...,0.0,0.0,0.0,0.0,0.0,1.0,toxic,5
001956c382006abd,i m sorry i m sorry i screwed around with some...,0.0,0.0,0.0,0.0,0.0,1.0,toxic,5
002264ea4d5f2887,why can t you believe how fat artie is did yo...,0.0,0.0,0.0,0.0,0.0,1.0,toxic,5
005f59485fcddeb0,sorry puck but no one ever said dick was numb...,0.0,0.0,0.0,0.0,0.0,1.0,toxic,5


In [7]:
#use stratified sampling to ensure that sampled dataset is representative of the entire population
X_train, X_val, y_train, y_val= train_test_split(df.index.values, df.label_value.values, random_state = 42, stratify = df['label'].values)


AttributeError: 'DataFrame' object has no attribute 'label_value'

In [246]:
#create set column
#locate training and val samples
df['sample_type'] = ['none']*df.shape[0]

df.loc[X_train, 'sample_type'] = 'train'
df.loc[X_val, 'sample_type'] = 'val'

In [247]:
#check stratified sampling values

df.groupby(['label', 'label_value', 'sample_type']).count()

comment_text  ...  toxic
label         label_value sample_type                ...       
identity_hate 0           train                  40  ...     40
                          val                    14  ...     14
insult        1           train                 226  ...    226
                          val                    75  ...     75
obscene       2           train                 238  ...    238
                          val                    79  ...     79
threat        4           train                  17  ...     17
                          val                     5  ...      5
toxic         5           train                4249  ...   4249
                          val                  1417  ...   1417

[10 rows x 7 columns]

In [248]:
!pip install transformers==3.5

In [249]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [250]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)


In [251]:
#encode train data
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.sample_type=='train'].comment_text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    max_length = 256,
    padding = True,
    truncation = True,
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.sample_type=='train'].label_value.values)



In [252]:
#encode validation data
encoded_data_val = tokenizer.batch_encode_plus(
    df[df.sample_type=='val'].comment_text.values, 
   add_special_tokens=True, 
    return_attention_mask=True, 
    max_length = 256,
    padding = True,
    truncation = True,
    return_tensors='pt'
)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.sample_type=='val'].label_value.values)

In [253]:
# Convert all inputs and labels into torch tensors
#BERT model required datatype 
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [254]:
print(len(dataset_train))
print(len(dataset_val))

4770
1590


In [255]:
from transformers import BertForSequenceClassification

In [256]:
 model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                       num_labels=len(possible_labels),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [257]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [258]:
batch_size = 32

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [259]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [260]:
optimizer = AdamW(model.parameters())

In [261]:
epochs = 10

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [262]:
from sklearn.metrics import f1_score

In [263]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [264]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [265]:
import random
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [266]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [267]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [269]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 0.5326458472510179
Validation loss: 0.4970031551271677
F1 Score (Weighted): 0.8399223614501173



Epoch 2
Training loss: 0.4851704913377762
Validation loss: 0.46577891454100606
F1 Score (Weighted): 0.8399223614501173



Epoch 3
Training loss: 0.47710482935110726
Validation loss: 0.48643613487482074
F1 Score (Weighted): 0.8399223614501173



Epoch 4
Training loss: 0.48505168572068214
Validation loss: 0.46631444901227953
F1 Score (Weighted): 0.8399223614501173



Epoch 5
Training loss: 0.4734137836595376
Validation loss: 0.47372164085507396
F1 Score (Weighted): 0.8399223614501173



Epoch 6
Training loss: 0.47302221131821476
Validation loss: 0.46492910400033
F1 Score (Weighted): 0.8399223614501173



Epoch 7
Training loss: 0.476940329571565
Validation loss: 0.4598277512192726
F1 Score (Weighted): 0.8399223614501173



Epoch 8
Training loss: 0.4653632979094982
Validation loss: 0.4685015004873276
F1 Score (Weighted): 0.8399223614501173



Epoch 9
Training loss: 0.4637651174267133
Validation loss: 0.46462376579642295
F1 Score (Weighted): 0.8399223614501173



Epoch 10
Training loss: 0.463958105246226
Validation loss: 0.45730825573205947
F1 Score (Weighted): 0.8399223614501173



In [272]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [273]:
accuracy_per_class(predictions, true_vals)

Class: identity_hate
Accuracy: 4/14

Class: insult
Accuracy: 2/75

Class: obscene
Accuracy: 2/79

Class: threat
Accuracy: 4/5

Class: toxic
Accuracy: 371/1417



In [274]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [275]:
model_save_name = 'classifierII.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 
torch.save(model.state_dict(), path)